# ConceptNetist lisaküsimuste saamine

In [ ]:
# tõlgime sõna inglise keelde
# leiame seosed ConceptNetist
# tõlgime seosed eesti keelde
# teeme erinevatest seoste tüüpidest küsimused
# ühendame sõna ja küsimuse
# lisame juurde alamsõnede seosed -- tegemata

## importimine

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/baka')

Mounted at /content/drive


In [ ]:
!pip install requests

In [ ]:
!pip install estnltk==1.7.3
import estnltk
from estnltk import Text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.3 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd

## andmete failist lugemine

In [ ]:
df = pd.read_csv('mängu_andmed_mineraalid.csv')

In [ ]:
words = df["Sõna"].tolist() #len = 220

## Sõnade tõlkimise funktsioon

In [ ]:
import requests

# https://translate.ut.ee/API
# https://github.com/TartuNLP/translation-api

import time

session = requests.Session()

def translate_text(text, src, tgt, url="https://api.tartunlp.ai/translation/v2"):
    headers = {
        "x-api-key": "public",
        "Content-Type": "application/json"
    }
    data = {
        "text": text,
        "src": src,
        "tgt": tgt,
        "application": "test-application"
    }

    try:
        response = session.post(url, json=data, headers=headers)
        response.raise_for_status()  # Raise error for bad responses
        return response.json().get("result")
    except requests.exceptions.RequestException as e:
        print(f"Translation error: {e}")
        return None



In [ ]:
from estnltk.vabamorf.morf import synthesize
def correct_word_form(word, vabamorf):
  word = word.split(".")
  word = word[0]
  sõna = ""
  for w in word.split():
    synthesized = synthesize(w, vabamorf)
    if synthesized:
        sõna += synthesized[0] + " "
    else:
        return

  if sõna == "":
    return word
  return sõna.strip()

## Küsimuste genereerimine

In [ ]:
def generate_question(words, relation):
  verb = ""
  noun = ""

  words = translate_text(words, "en", "et")
  text = Text(words).tag_layer().morph_analysis

  if relation == "CapableOf": # kasutan sõnapaare mis on kujul VERB + NOUN
    for i in range(len(text.lemma)):
      wordPOS = text.partofspeech[i]
      if wordPOS[0] == 'V':
        if correct_word_form(text.lemma[i][0], "b"):
          verb = correct_word_form(text.lemma[i][0], "b")
        else:
          verb = text.lemma[i][0]
      elif wordPOS[0] == "S":
        noun = correct_word_form(text.lemma[i][0], "pl p")
      else:
        continue
    if not noun or not verb:
      return
    else:
      question = f"Kas see mineraal {verb} {noun}?"
  elif relation == "AtLocation": # kasutan ainult ühte NOUNi
    for i in range(len(text.lemma)):
      wordPOS = text.partofspeech[i]
      if wordPOS[0] == "S":
        noun = correct_word_form(text.lemma[i][0], "sg ad")
      else:
        continue
    if not noun:
      return
    else:
      question = f"Kas seda mineraali võib leiduda {noun}?"
  elif relation == "HasProperty":
    for i in range(len(text.lemma)):
      wordPOS = text.partofspeech[i]
      if wordPOS[0] == "S":
        noun = correct_word_form(text.lemma[i][0], "sg n")
      else:
        continue
    if not noun:
      return
    else:
      question = f"Kas see mineraal on {noun}?"
  elif relation == "HasA":
    for i in range(len(text.lemma)):
      wordPOS = text.partofspeech[i]
      if wordPOS[0] == "S":
        noun = correct_word_form(text.lemma[i][0], "sg n")
      else:
        continue
    if not noun:
      return
    else:
      question = f"Kas sellel mineraalil on {noun}?"
  elif relation == "UsedFor":
    for i in range(len(text.lemma)):
      wordPOS = text.partofspeech[i]
      if wordPOS[0] == 'V':
        if correct_word_form(text.lemma[i][0], "ks"):
          verb = correct_word_form(text.lemma[i][0], "ks")
        else:
          verb = text.lemma[i][0]
      elif wordPOS[0] == "S":
        noun = correct_word_form(text.lemma[i][0], "pl tr")
      else:
        continue
    if not noun or not verb:
      return
    else:
      question = f"Kas seda mineraali kasutatakse {verb} {noun}?"
  else:
    return


  return translate_text(question, "et", "et")

## Seoste saamine ConceptNetist

In [ ]:
import requests
import time

# Create a session for efficiency
session = requests.Session()
session.headers.update({"Connection": "keep-alive"})  # Keep the session alive

def get_conceptnet_relationships(word):
    base_url = "http://api.conceptnet.io"
    start_url = f"/query?node=/c/en/{word}"
    min_weight = 0.9  # Set minimum weight threshold
    target_relations = {"CapableOf", "AtLocation", "HasProperty", "HasA", "UsedFor"}
    filtered_edges = []
    questions = set()

    # Loop through pages
    while start_url:
        try:
            response = session.get(base_url + start_url, timeout=10)  # Use session for better performance
            response.raise_for_status()  # Ensure successful request
            data = response.json()

            for edge in data.get("edges", []):
                relation = edge["rel"]["label"]
                weight = edge.get("weight", 0)

                if relation in target_relations and weight > min_weight:
                    filtered_edges.append(edge)

            start_url = data.get("view", {}).get("nextPage")  # Get next page URL

            time.sleep(0.5)  # Small delay to avoid rate limiting

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {word}: {e}")
            break  # Stop on error

    # Sort edges by weight in descending order and select the top 30
    filtered_edges = sorted(filtered_edges, key=lambda x: x.get("weight", 0), reverse=True)[:35]

    # Collect filtered relationships
    for edge in filtered_edges:
        relation = edge["rel"]["label"]
        target = edge["end"]["label"]
        if all(part != word for part in target.split()):
            question = generate_question(target, relation)
            questions.add(question)

    return questions


## main run ja faili kirjutamine

In [ ]:
word_questions = {}
for word in words:
  wordENG = translate_text(word, "et", "en")
  word_questions[word] = get_conceptnet_relationships(wordENG)

In [ ]:
import json

#Remove None values and convert sets to lists
word_questions = {word: list({q for q in questions if q is not None}) for word, questions in word_questions.items()}

# Save to file
with open("word_questions_mineraalid.txt", "w", encoding="utf-8") as file:
    json.dump(word_questions, file, ensure_ascii=False, indent=4)

## loeme failid sisse ja leiame alammõisted

In [ ]:
import ast

hyponym_map = {}  # Dictionary to store results

with open("word_questions_mineraalid.txt", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        line = line.replace("set()", "[]")  # Replace `set()` with `[]` for eval
        print(line)
        my_list = ast.literal_eval(line)  # Convert string to list

        key = tuple(my_list[0])  # Convert list to a tuple (hashable)
        hyponym_map[key] = my_list[1]  # Assign the second element

print(hyponym_map)



{


SyntaxError: '{' was never closed (<unknown>, line 1)

In [ ]:
with open("word_questions_mineraalid.txt", "r", encoding="utf-8") as file:
    questions_data = json.loads(file.read())

In [ ]:
questions_data

{'adulaar': [],
 'aktiivmuld': [],
 'aktinoliit': [],
 'aktsessoorne mineraal': [],
 'albiit': [],
 'alumosilikaat': [],
 'amfibool': [],
 'amiant': [],
 'titaanoksiid': [],
 'andesiin': [],
 'anglesiit': [],
 'anhüdriit': [],
 'ankeriit': [],
 'anortiit': [],
 'antofülliit': [],
 'apatiit': [],
 'aragoniit': [],
 'auripigment': [],
 'asbest': [],
 'askariit': [],
 'asuriit': [],
 'barüüt': [],
 'berüll': [],
 'biotiit': [],
 'bismutiniit': [],
 'booraks': [],
 'diaspoor': [],
 'diopsiid': [],
 'dolomiit': [],
 'epidoot': [],
 'fenakiit': [],
 'flogopiit': [],
 'fluoriit': [],
 'gagaat': [],
 'galeniit': [],
 'glaukoniit': [],
 'vilgukivi': [],
 'grafiit': ['Kas seda mineraalt võib pliiatsis leiduda?'],
 'götiit': [],
 'haliit': [],
 'idokraas': [],
 'ilmeniit': [],
 'kaaliumpäevakivi': [],
 'kainiit': [],
 'kaltsiit': [],
 'kaoliniit': [],
 'karnalliit': [],
 'kassikuld': [],
 'kassiteriit': [],
 'kinaver': [],
 'kleveiit': [],
 'kloriit': [],
 'kobaltiin': [],
 'kokoliit': [],
 'koos

In [ ]:
# vaatame paljud küsimused korduvad
unique_questions = set()
all_questions = []
duplicates = 0
example = ""

for words, questions in questions_data.items():
    all_questions.extend(questions)  # Add all questions to the list
    for question in questions:
        if question in unique_questions:
            duplicates += 1  # Count duplicates
            example += question
        else:
            unique_questions.add(question)  # Add to unique set

# Print the number of unique and duplicate questions
print(f"Unique questions: {len(unique_questions)}")
print(f"Total questions: {len(all_questions)}")
print(f"Duplicate questions: {duplicates}")
#print(example)

Unique questions: 4
Total questions: 4
Duplicate questions: 0


In [ ]:
# käin läbi kõik df["Sõna"] väljad ja vaatan, kas antud sõnale on mõni küsimus juurde lisada
# kui on kontrollin, et sellist küsimust juba ei ole
# kui küsimust juba ei ole, lisan selle küsimuse df ja lisan sõnale seoseks 1 ja kõigile alammõistetele ka 1, muidu 0
# kui küsimus on juba olemas, siis lisan seosed, mida veel ei ole

# Funktsioon küsimuste lisamiseks
def update_questions(df, questions_data, hyponym_map):
    for word, questions in questions_data.items():
        if questions:  # Ensures questions is not empty
            updated_words = set()

            # Find main words and hyponyms
            for mainword, hyponyms in hyponym_map.items():
                if word in mainword:
                    updated_words.add(word)
                    updated_words.update(hyponyms)

            # Process each updated word
            for question in questions:
                if question not in df.columns:
                    df[question] = 0  # Create a new column if it doesn't exist

                # Update the values: set 1 for matching "Sõna" values
                df.loc[df["Sõna"].isin(updated_words) & df[question].isna(), question] = 1

    return df

# Rakendame funktsiooni
updated_df = update_questions(df, questions_data, hyponym_map)



In [ ]:
updated_df

,Sõna,Kas seda mineraali on amfiboliidis?,Kas seda mineraali on arseenis?,Kas seda mineraali on baariumsulfaadis?,Kas seda mineraali on fosforväetises?,Kas seda mineraali on kaltsedonis?,Kas seda mineraali on kondiportselanis?,Kas seda mineraali on labradoriidis?,Kas seda mineraali on liivakivis?,Kas seda mineraali on lubjakivis?,...,Kas tegu on põldpaoga?,Kas tegu on pürokseeniga?,Kas tegu on silikaatmineraaliga?,Kas tegu on uraanipigimaagiga?,Synonyms,tähtsus,Kas seda mineraalt võib pliiatsis leiduda?,Kas see mineraalaine võib lambis olla?,Kas seda mineraali võib juveelipoes leiduda?,Kas seda mineraalt leidub käekellas?
0,adulaar,0,0,0,0,0,1,0,0,0,...,1,0,0,0,adulaar,1,0,0,0,0
1,aktiivmuld,0,0,0,0,0,0,0,0,0,...,0,0,0,0,"aktiivmuld, pleekmuld",4,0,0,0,0
2,aktinoliit,1,0,0,0,0,0,0,0,0,...,0,0,0,0,aktinoliit,2,0,0,0,0
3,aktsessoorne mineraal,0,0,0,0,0,0,0,0,0,...,0,0,0,0,NaN,2,0,0,0,0
4,albiit,0,0,0,0,0,1,0,0,0,...,1,0,0,0,albiit,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,vermikuliit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,vermikuliit,2,0,0,0,0
109,vivianiit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,vivianiit,2,0,0,0,0
110,volframiit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,"volframiit, volframimaak",2,0,0,0,0
111,vollastoniit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,vollastoniit,2,0,0,0,0


In [ ]:
#updated_df = updated_df.drop(columns=["Unnamed: 1"])

In [ ]:
#updated_df = updated_df.drop(columns=["Kas see loom on ?"])

In [ ]:
updated_df

,Loom,Kas seda looma võib leiduda avamaastikus?,Kas seda looma võib leiduda avameres?,Kas seda looma võib leiduda avaspesas?,Kas seda looma võib leiduda järves?,Kas seda looma võib leiduda jões?,Kas seda looma võib leiduda lähistroopikas?,Kas seda looma võib leiduda mererannikus?,Kas seda looma võib leiduda tiigis?,Kas see loom ammub?,...,Kas seda looma võib järve ääres leiduda?,Kas see loom võib olla messil?,Kas see loom võib olla karnevalil?,Kas seda looma võib leida alusmetsast?,Kas seda looma võib väljakul leiduda?,Kas see loom võib mäel olla?,Kas see loom võib olla urus?,Kas see loom võib roomajal olla?,Kas see loom võib olla džunglis?,Kas seda looma võib leiduda lemmikloomakaupluses?
0,ameerika paalia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,arvicola terrestris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,campbelli käpikhamstrik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dalli lammas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Jack Russel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,kalifornia pringel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,aafrika elevant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,aafrika pühvel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,aafrika tsiibet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,aaga,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
updated_df.to_csv('game_data_minerals_final.csv', index=False)

In [ ]:
#viimased katsetused

In [ ]:
df = pd.read_csv('game_data_minerals_final.csv')

In [ ]:
for col in df.columns: # veerud: Synonyms, tähtsus
    print(col)

Sõna
Kas seda mineraali on amfiboliidis?
Kas seda mineraali on arseenis?
Kas seda mineraali on baariumsulfaadis?
Kas seda mineraali on fosforväetises?
Kas seda mineraali on kaltsedonis?
Kas seda mineraali on kondiportselanis?
Kas seda mineraali on labradoriidis?
Kas seda mineraali on liivakivis?
Kas seda mineraali on lubjakivis?
Kas seda mineraali on magneesiumis?
Kas seda mineraali on mangaanis?
Kas seda mineraali on mikaniidis?
Kas seda mineraali on muldmetallis?
Kas seda mineraali on orgaanilises värvaines?
Kas seda mineraali on peridotiidis?
Kas seda mineraali on perliitmalmis?
Kas seda mineraali on põldpaoglasuuris?
Kas seda mineraali on pürokseniidis?
Kas seda mineraali on rullpitsatis?
Kas seda mineraali on tsirkoontulises?
Kas seda mineraali on värvis?
Kas see mineraal on alumiiniumi tootmisel kasutatav kergesti sulav klaasi läikega mineraal?
Kas see mineraal on alumosilikaatide hulka kuuluv kristalliseerunud mineraal?
Kas see mineraal on alumosilikaatide hulka kuuluv vett sisa

In [ ]:
df.to_csv('game_data_minerals_final.csv', index=False)

In [ ]:
question = "Kas see loom võib roomajal olla?"
df[df[question] == 1]

,Sõna,Kas seda looma võib leiduda avamaastikus?,Kas seda looma võib leiduda avameres?,Kas seda looma võib leiduda avaspesas?,Kas seda looma võib leiduda järves?,Kas seda looma võib leiduda jões?,Kas seda looma võib leiduda lähistroopikas?,Kas seda looma võib leiduda mererannikus?,Kas seda looma võib leiduda tiigis?,Kas see loom ammub?,...,Kas seda looma võib järve ääres leiduda?,Kas see loom võib olla messil?,Kas see loom võib olla karnevalil?,Kas seda looma võib leida alusmetsast?,Kas seda looma võib väljakul leiduda?,Kas see loom võib mäel olla?,Kas see loom võib olla urus?,Kas see loom võib roomajal olla?,Kas see loom võib olla džunglis?,Kas seda looma võib leiduda lemmikloomakaupluses?
32,allosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
84,brahhiosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
88,brontosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103,dinosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
104,diplodookus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
108,dromeosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
152,gigantosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
156,gorgosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
214,hiid-ogasaba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
278,iguaanodon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
row = df[df["Sõna"] == "õun"]  # Leia rida, kus "Sõna" on "kana"

if not row.empty:  # Kontrollime, et rida on leitud
    yes_columns = row.iloc[0][row.iloc[0] == 1].index.tolist()  # Leia veerud, kus väärtus on 1
    print("Jaatavalt vastatud küsimuste veerud:", yes_columns)
else:
    print("Viga: Sõna 'kana' ei leitud andmestikust.")

for col in yes_columns: # veerud: Synonyms, tähtsus
    print(col)


Jaatavalt vastatud küsimuste veerud: ['Kas see taim on punane, kollane v. roheline õunapuu vili?', 'Kas sellest taimest saab teha puuviljajooki?', 'Kas sellest taimest saab teha puuviljakompotti?', 'Kas sellest taimest saab teha puuviljakonservi?', 'Kas sellest taimest saab teha puuviljakooki?', 'Kas sellest taimest saab teha puuviljakreemi?', 'Kas sellest taimest saab teha puuviljapudingit?', 'Kas sellest taimest saab teha puuviljapüreed?', 'Kas sellest taimest saab teha puuviljasalatit?', 'Kas sellest taimest saab teha puuviljasiirupit?', 'Kas sellest taimest saab teha puuviljatorti?', 'Kas sellest taimest saab teha puuviljaveini?', 'Kas sellest taimest saab teha segakompotti?', 'Kas sellest taimest saab teha sukaadi?', 'Kas sellest taimest saab teha vesihoidist?', 'Kas sellest taimest saab teha õunakastet?', 'Kas sellest taimest saab teha õunakeedist?', 'Kas sellest taimest saab teha õunakompotti?', 'Kas sellest taimest saab teha õunakooki?', 'Kas sellest taimest saab teha õunakreem